In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from tqdm import tqdm
import re
import csv

In [2]:
def get_all_urls_from_website(website_url):
    
    """this function get all urls from proper website"""
    
    driver = webdriver.Edge()
    driver.get(website_url)
    wait = WebDriverWait(driver, 8)
    links, all_urls = [], []
    
    # Scroll down the page approximately 20 times
    for _ in range(20):
        
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)

    for i in range(2, 130):
        
        xpath = f'//*[@id="app"]/div/main/div[2]/div/div[3]/div/div/div[{i}]/a'
        
        try:
            element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
            link = element.get_attribute('href')
            links.append(link)
            
        except NoSuchElementException:
            print(f'Element not found: {xpath}')
            
    all_urls= '\n'.join(links).split('\n')
    driver.close()
    column_titles = [ 'Links']
    df = pd.DataFrame(all_urls, columns=column_titles)
    
    df.to_csv('Tripsome_az_all_urls.csv', index=False)
    
    return df

In [3]:
data1=get_all_urls_from_website("https://tripsome.az/az/categories/all")

,Links
0,https://tripsome.az/az/activities/f1b7455/futs...
1,https://tripsome.az/az/activities/f0c434d/cine...
2,https://tripsome.az/az/activities/9c50227/snay...
3,https://tripsome.az/az/activities/f180dfe/kart...
4,https://tripsome.az/az/activities/8c41954/mix-...
...,...
123,https://tripsome.az/az/activities/fc916e8/powe...
124,https://tripsome.az/az/activities/1040111/lati...
125,https://tripsome.az/az/activities/3c75f2b/elek...
126,https://tripsome.az/az/activities/7e1a6cf/kera...


In [33]:
def activity_information(url):
    
    """this function collect a comprehensive list of physical activities that you want to suggest to users."""
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    date, datelist, available_dates = [], [], []
    my_dict={}
    
    category=soup.find_all('div', attrs = {'class':'c-activity-page-category'})[0].text.replace("\n","").split(":")[1].strip()
    activity_name=soup.find_all('div', attrs = {'class':'c-activity-page-title'})[0].text
    price=soup.find_all('div', attrs = {'class':'c-activity-price'})[0].text.replace("\n","").split(" ")[1]
    age_range=soup.find_all('div', attrs = {'class':'c-activity-page-age text-secondary'})[0].text.replace(".\n","").strip().split("+")[0].strip()+"+"
    activity_description=soup.find_all('div', attrs = {'class':'c-activity-section-info'})[0].text
    selected = soup.find_all('activity-date-time')
    
    
    for x in selected:
        
        date.append(str(x))

    def week_func(n_day):
        
        gun=''
        if n_day=='sa':
            gun="Şən."
        elif n_day=='su':
            gun="Baz"
        elif n_day=='m':
            gun="Baz.E."
        elif n_day=='tu':
            gun="Çərş.A."
        elif n_day=='w':
            gun="Çərş."
        elif n_day=='th':
            gun="Cüm.A."
        elif n_day=='f':
            gun="Cüm."

        return gun


    def month_func(n_month):
        ay=''
        if n_month=='04':
            ay="Apr."
        elif n_month=='05':
            ay="May"
        elif n_month=='06':
            ay="Iyn."
        elif n_month=='07':
            ay="Iyl."
        elif n_month=='08':
            ay="Avq."
        elif n_month=='09':
            ay="Sen."
        elif n_month=='10':
            ay="Okt."
        elif n_month=='11':
            ay="Noy." 
        elif n_month=='12':
            ay="Dek."
        return ay

    json_string = date[0]

    # Extract the JSON data from the string
    json_data = json.loads(json_string.split(":terms=\'")[1].split("\'>")[0])


    for i in json_data:
        lst=[]
        lst += [i['firstDay'][8:10]+" "+month_func(i['firstDay'][5:7])]
        lst += [i['lastDay'][8:10]+" "+month_func(i['lastDay'][5:7])]
        lst += [i['time_from']]
        lst += [i['time_to']]

        week=['m','tu','w','th','f','sa','su']


        for k in week:
            if i[k]==1:
                lst += [week_func(k)]

        datelist+= [lst]

    for i in datelist:
        lst = []
        lst += [i[4] + ' - ' + i[-1] + ', ' + i[2][:5]+ '-' + i[3][:5]+' ('+ i[0][0:]+ ' - '+ i[1][0:] + ')' ]
        available_dates += lst 
        
    activity_information_dict={"Category":category,"Activity_name":activity_name,"Price":price,"Age_range":age_range,"Activity_description":activity_description,"Available_dates":available_dates}
    
    return activity_information_dict

100%|████████████████████████████████████████████████████████████████████████████████| 128/128 [02:06<00:00,  1.01it/s]


In [ ]:
list_of_activity_information=[]

for i in tqdm(range(len(data1['Links']))):
    list_of_activity_information.append(activity_information(data1['Links'][i]))      

In [39]:
keys = list_of_activity_information[0].keys()

filename = open("list_of_activity_information.csv", "w",encoding="utf-8")
a = csv.DictWriter(filename,keys)
a.writeheader()
a.writerows(list_of_activity_information)
filename.close()

,Category,Activity_name,Price,Age_range,Activity_description,Available_dates
0,İdman və Döyüş Sənəti,Futsal Meetup (hər həftə),7,14+,Biletə daxildir:\r\n\r\nOyun üçün xalça sahə\r...,"['Cüm.A. - Cüm.A., 22:00-23:00 (04 May - 28 De..."
1,"Tamaşa, Teatr, Şoular","Cinema Fulldome - ""Kəşf Edilməyən Dünyalar""",3,3+,\r\nMüddət: 30 dəqiqə\r\nTarix: Hər şənbə və b...,"['Şən. - Baz, 17:00-17:30 (29 Apr. - 25 Iyn.)'..."
2,İdman və Döyüş Sənəti,"Snayper - Nəfəs, Dəqiqlik və Atəş",14,7+,Qiymətə daxildir\r\n\r\nSilahla düzgün davranı...,"['Çərş.A. - Cüm., 12:00-20:00 (28 Apr. - 29 De..."
3,İdman və Döyüş Sənəti,Kartinq Sür və Formula 1 həyəcanını hiss et,15,10+,Qiymətə daxildir:\r\n\r\nTəhlükəsizlik brifinq...,"['Baz.E. - Baz, 12:00-00:00 (28 Apr. - 31 Dek.)']"
4,İdman və Döyüş Sənəti,Mix - Snayper və Avtomat Airsoft,18,7+,Qiymətə daxildir\r\n\r\nSilahla düzgün davranı...,"['Çərş.A. - Baz, 12:00-20:00 (28 Apr. - 31 Dek..."
...,...,...,...,...,...,...
123,Musiqi və Rəqs,"Latın Amerika Rəqsləri - Salsa, Bachata, Kizom...",30,14+,"Latın Amerika Rəqsləri\r\nFərdi, qrup, cütlük ...","['Baz.E. - Şən., 19:00-20:00 (28 Apr. - 30 Dek..."
124,Turlar və Səyahət,Elektrik Velosipedi Turu,100,14+,Tur elektrik velosipedləri ilə həyata keçirili...,"['Baz - Baz, 07:00-18:00 (30 Apr. - 31 Dek.)']"
125,İncəsənət və Sənətkarlıq,Keramika -Gil Yapma Sənəti,30,5+,Studioda keramika və rəngkarlıq dərsləri keçir...,"['Baz.E. - Baz, 10:00-19:00 (28 Apr. - 31 Dek...."
126,İdman və Döyüş Sənəti,Tennis Masterklassı,10,5+,Sizi tennis üzrə Azərbaycan Çempionatının qali...,"['Çərş. - Çərş., 09:00-11:00 (03 May - 27 Dek.)']"


In [5]:
data = pd.read_csv('list_of_activity_information.csv')
data

,Category,Activity_name,Price,Age_range,Activity_description,Available_dates
0,İdman və Döyüş Sənəti,Futsal Meetup (hər həftə),7,14+,Biletə daxildir:\r\n\r\nOyun üçün xalça sahə\r...,"['Cüm.A. - Cüm.A., 22:00-23:00 (04 May - 28 De..."
1,"Tamaşa, Teatr, Şoular","Cinema Fulldome - ""Kəşf Edilməyən Dünyalar""",3,3+,\r\nMüddət: 30 dəqiqə\r\nTarix: Hər şənbə və b...,"['Şən. - Baz, 17:00-17:30 (29 Apr. - 25 Iyn.)'..."
2,İdman və Döyüş Sənəti,"Snayper - Nəfəs, Dəqiqlik və Atəş",14,7+,Qiymətə daxildir\r\n\r\nSilahla düzgün davranı...,"['Çərş.A. - Cüm., 12:00-20:00 (28 Apr. - 29 De..."
3,İdman və Döyüş Sənəti,Kartinq Sür və Formula 1 həyəcanını hiss et,15,10+,Qiymətə daxildir:\r\n\r\nTəhlükəsizlik brifinq...,"['Baz.E. - Baz, 12:00-00:00 (28 Apr. - 31 Dek.)']"
4,İdman və Döyüş Sənəti,Mix - Snayper və Avtomat Airsoft,18,7+,Qiymətə daxildir\r\n\r\nSilahla düzgün davranı...,"['Çərş.A. - Baz, 12:00-20:00 (28 Apr. - 31 Dek..."
...,...,...,...,...,...,...
123,Musiqi və Rəqs,"Latın Amerika Rəqsləri - Salsa, Bachata, Kizom...",30,14+,"Latın Amerika Rəqsləri\r\nFərdi, qrup, cütlük ...","['Baz.E. - Şən., 19:00-20:00 (28 Apr. - 30 Dek..."
124,Turlar və Səyahət,Elektrik Velosipedi Turu,100,14+,Tur elektrik velosipedləri ilə həyata keçirili...,"['Baz - Baz, 07:00-18:00 (30 Apr. - 31 Dek.)']"
125,İncəsənət və Sənətkarlıq,Keramika -Gil Yapma Sənəti,30,5+,Studioda keramika və rəngkarlıq dərsləri keçir...,"['Baz.E. - Baz, 10:00-19:00 (28 Apr. - 31 Dek...."
126,İdman və Döyüş Sənəti,Tennis Masterklassı,10,5+,Sizi tennis üzrə Azərbaycan Çempionatının qali...,"['Çərş. - Çərş., 09:00-11:00 (03 May - 27 Dek.)']"


In [10]:
def find_available_activity():
    
    """ this function suggest activities to users based on their preferences"""
    
    category=input("please choose a category:")
    price=input("please choose price range:")
    price_list=price.split("-")
    
    k=0
    mylist=[]

    for i in range(len(data)):
        
        if category==data['Category'][i] and int(price_list[0])<=data["Price"][i]<=int(price_list[1]):
            k+=1
            mylist += [i]
            
    if k<=2 and k!=0:
        
        print(f"\nAccording to your request found {len(mylist)} activities.\n")
        for k in mylist:
            print("Activity_name:",data["Activity_name"][k],"\n","Price:",str(data["Price"][k])+"AZN","\n","Age_range:",data['Age_range'][k],"\n",'Activity_description:',data['Activity_description'][k],"\n","Available_dates:",data['Available_dates'][k], '\n',"access for activity:","\n")
            print("\033[1m***********************************************************************************\033[0m","\n")
            
    elif k>2:
        
        special_request=input("A few activities have found according to your request .Do you have special request? Yes or No:")
        
        if special_request=="Yes":
            
            special_request=input("please add your special requests:")
            special_request_list=special_request.split(",")
            
            for i in range(len(data)):
                
                if category==data['Category'][i] and int(price_list[0])<=data["Price"][i]<=int(price_list[1]):
                    list_ = []
                    for j in special_request_list:
                        matches = re.findall(f"{j.lower()}", (data['Activity_description'][i]).lower())
                        list_ += matches

                    if len(list_)>0:
                        k+=1
                        mylist += [[i] + list(set(list_))]
            mylist2=[]
            for s in mylist:
                if type(s)!=int:
                    mylist2.append(s)

            max_len_index = max(range(len(mylist2)), key=lambda i: len(mylist2[i]))
            percentage = (len(mylist2[max_len_index])-1)/(len(special_request_list))
            count = sum((1 for k in mylist2 if len(k)==len(mylist2[max_len_index])))
            if percentage!=1:
                print(f"\nAccording to your request  found {round(percentage*100,2)} % accuracy {count} activities.\n")
            else:
                print(f"\nAccording to your request found {count} activities.\n")
            for k in mylist2:
                if len(k)==len(mylist2[max_len_index]):
                    print("Activity_name:",data["Activity_name"][k[0]],"\n","Price:",str(data["Price"][k[0]])+"AZN","\n","Age_range:",data['Age_range'][k[0]],"\n",'Activity_description:',data['Activity_description'][k[0]],"\n","Available_dates:",data['Available_dates'][k[0]], '\n',"access for activity:","\n")
                    print("\033[1m***********************************************************************************\033[0m","\n")
        else:
            print(f"\nAccording to your request found {len(mylist)} activities.\n")
            for k in mylist:
                print("Activity_name:",data["Activity_name"][k],"\n","Price:",str(data["Price"][k])+"AZN","\n","Age_range:",data['Age_range'][k],"\n",'Activity_description:',data['Activity_description'][k],"\n","Available_dates:",data['Available_dates'][k], '\n',"access for activity:","\n")
                print("\033[1m***********************************************************************************\033[0m","\n")
    
    else:
        print("There is no activity to your request")

In [9]:
find_available_activity()

please choose a category:İdman və Döyüş Sənəti
please choose price range:7-8

According to your request found 2 activities.

Activity_name: Futsal Meetup (hər həftə) 
 Price: 7AZN 
 Age_range: 14+ 
 Activity_description: Biletə daxildir:

Oyun üçün xalça sahə
Standardlara uyğun top
Komandalar üçün 2 rəngdə uniforma (naketka)
Oyundan əvvəl və sonra istifadə üçün dəyişmə otağı
İsti və soyuq su ilə təmin olunmuş hamam/duş
Oyundan sonra limonlu çay

Qiymət: 7AZN
Dil: Azərbaycan, Rus, İngilis, Ərəb dillərini bilirik.
Ləğv qaydası: Xidmətin baş tutmasına 96 saat və ya daha çox müddət qaldıqda bilet geri qaytarılarsa və ya dəyişdirilərsə, bilet dəyərinin 20%-i tutulur. Xidmətin baş tutmasına 96 saat və ya daha az müddət qaldıqda bilet geri qaytarılmır və ya dəyişdirilmir.
Ünvan: 178 saylı orta məktəbin arxası, Nərimanov, saatlı bina ilə üzbəüz
Startup, Biznes, İdman, Texnologiya, Kripto, Əlaqələr, Peşəkar və həvəskarlar ilə birlikdə Limonlu Çay Söhbəti! Hamısı birlikdə hər həftə Startup Futsa